In [1]:
import sys
sys.path.append("../")
from plotting import CandlePlot
import pandas as pd
from technical.patterns import apply_patterns

In [2]:
df = pd.read_pickle("../data/GBP_JPY_H1.pkl")

In [3]:
df_an = df[["time", "mid_o", "mid_h","mid_l", "mid_c" ]].copy()

In [4]:
df_an.tail()

,time,mid_o,mid_h,mid_l,mid_c
59622,2024-12-30 19:00:00+00:00,197.074,197.112,196.911,197.045
59623,2024-12-30 20:00:00+00:00,197.044,197.077,196.729,196.754
59624,2024-12-30 21:00:00+00:00,196.754,196.892,196.688,196.880
59625,2024-12-30 22:00:00+00:00,196.872,196.938,196.786,196.802
59626,2024-12-30 23:00:00+00:00,196.818,197.054,196.772,197.012


In [5]:
direction = df_an.mid_c - df_an.mid_o 
body_size = abs(direction)
direction = [1 if x>= 0 else -1 for x in direction]
full_range = df_an.mid_h - df_an.mid_l
body_perc = (body_size / full_range) * 100 
body_lower = df_an[["mid_c", "mid_o"]].min(axis=1)
body_upper = df_an[["mid_c", "mid_o"]].max(axis=1)
body_bottom_perc =( body_lower - df_an.mid_l) / full_range * 100
body_top_perc = 100 - (((df_an.mid_h - body_upper) / full_range) * 100)

In [6]:
df_an['body_lower'] = body_lower
df_an["body_upper"] = body_upper
df_an['body_bottom_perc'] = body_bottom_perc
df_an["body_top_perc"] = body_top_perc
df_an['body_per'] = body_perc

In [7]:
df_an = apply_patterns(df_an)

In [8]:
dfp = df_an.iloc[-50:]
cp = CandlePlot(dfp)
cp.show_plot(line_traces=[])

In [9]:
df_an.iloc[-6]

time                2024-12-30 18:00:00+00:00
mid_o                                 197.132
mid_h                                 197.292
mid_l                                 196.966
mid_c                                 197.074
body_lower                            197.074
body_upper                            197.132
body_bottom_perc                    33.128834
body_top_perc                       50.920245
body_per                            17.791411
Name: 59621, dtype: object